In [2]:
import os
import pandas as pd

csv_dir = '/home/ksaff/Desktop/ttyd/fine_tuning/llama_finetuning/10_datasets'
ddl_dir = '/home/ksaff/Desktop/ttyd/fine_tuning/fine_tune_dbs/ddls'

csv_names = sorted(os.listdir(csv_dir))[:10]
ddl_names = sorted(os.listdir(ddl_dir))
ddl_names.pop(2) #Del DDL10

'ddl10.sql'

In [4]:
# ONE DF

df = pd.read_csv('/home/ksaff/Desktop/sql/no_err_queries.csv')
new_columns = ['instruction', 'output']
df.columns = new_columns

text_col = []

for _, row in df.iterrows():
    prompt = 'Make SQLite query based on DDL and instruction.'
    instruction = row['instruction']
    response = row['output']

    text = (
        prompt
        + '### Instruction:\n'
        + instruction
        + '### Response:\n'
        + response
    )

    text_col.append(text)

df['text'] = text_col

df.to_csv('/home/ksaff/Desktop/ttyd/fine_tuning/dataset/train.csv')

In [8]:
import re

def rewrite_DDL(DDL):
    lines = DDL.split('\n')
    modified_text = "DDL: "
    for line in lines:
        if len(line) < 2:
            pass
        elif '/*' in line:
            pass
        elif 'DROP TABLE' in line:
            pass

        else:
            line = re.sub(r'(?<!NOT )\bNULL\b', '', line)
            modified_text += f'{line}\n'
            
    return modified_text

dfs = []

for csv_file, ddl_file in zip(csv_names, ddl_names):
    csv_path = os.path.join(csv_dir, csv_file)
    ddl_path = os.path.join(ddl_dir, ddl_file)

    df = pd.read_csv(csv_path)
    with open(ddl_path) as file:
        ddl = file.read()
    ddl = rewrite_DDL(ddl)
    df = df.T.reset_index().T
    new_columns = ['instruction', 'output']
    df.columns = new_columns
    df.index = range(0, len(df))
    df['input'] = ddl
    dfs.append(df)

In [36]:
df = pd.concat(dfs, ignore_index=True)

In [10]:
text_col = []

for _, row in df.iterrows():
    prompt = 'Make SQLite query based on DDL and instruction.'
    instruction = row['instruction']
    input_query = row['input']
    response = row['output']

    text = (
        prompt
        + '### Instruction:\n'
        + instruction
        + '### Input:\n'
        + input_query
        + '### Response:\n'
        + response
    )

    text_col.append(text)

df['text'] = text_col

In [9]:
df = pd.read_csv('/home/ksaff/Desktop/ttyd/fine_tuning/dataset/train.csv')
df

,instruction,output,input,text
0,find the first name and last name of entities ...,"SELECT FIRST_NAME, LAST_NAME FROM JPK_ENTITY W...",DDL: CREATE TABLE 'JPK_HEADER'\n 'HEADER_ID...,Make SQLite query based on DDL and instruction...
1,show all headers sent after 1st January 2023,SELECT * FROM JPK_HEADER WHERE SEND_TIME > '20...,DDL: CREATE TABLE 'JPK_HEADER'\n 'HEADER_ID...,Make SQLite query based on DDL and instruction...
2,get all supplier numbers who have a purchase d...,SELECT SUPPLIER_NUMBER FROM VAT_PURCHASE WHERE...,DDL: CREATE TABLE 'JPK_HEADER'\n 'HEADER_ID...,Make SQLite query based on DDL and instruction...
3,fetch all purchases from supplier number 12345,SELECT * FROM VAT_PURCHASE WHERE SUPPLIER_NUMB...,DDL: CREATE TABLE 'JPK_HEADER'\n 'HEADER_ID...,Make SQLite query based on DDL and instruction...
4,list sales between April and July 2023,SELECT * FROM VAT_SALE WHERE ISSUE_DATE BETWEE...,DDL: CREATE TABLE 'JPK_HEADER'\n 'HEADER_ID...,Make SQLite query based on DDL and instruction...
...,...,...,...,...
1504,Retrieve the number of vehicles rented by cust...,SELECT COUNT(DISTINCT RR.VEHICLE_ID) FROM RENT...,DDL: CREATE TABLE 'VEHICLES' (\n 'VEHICLE_I...,Make SQLite query based on DDL and instruction...
1505,Find the average daily rental rate of all vehi...,SELECT AVG(RENTAL_RATE) FROM VEHICLES;,DDL: CREATE TABLE 'VEHICLES' (\n 'VEHICLE_I...,Make SQLite query based on DDL and instruction...
1506,Get the most recent rental record for a given ...,SELECT * FROM RENTAL_RECORDS WHERE VEHICLE_ID ...,DDL: CREATE TABLE 'VEHICLES' (\n 'VEHICLE_I...,Make SQLite query based on DDL and instruction...
1507,List the vehicles that have been rented for mo...,"SELECT V.VEHICLE_ID, SUM(RR.RETURN_DATE - RR.R...",DDL: CREATE TABLE 'VEHICLES' (\n 'VEHICLE_I...,Make SQLite query based on DDL and instruction...


In [40]:
text_col = []

for _, row in df.iterrows():
    prompt = 'Make SQLite query based on DDL and instruction.'
    instruction = row['instruction']
    input_query = row['input']
    response = row['output']

    text = (
        prompt
        + '### Instruction:\n'
        + instruction
        + '### Input:\n'
        + input_query
        + '### Response:\n'
        + response
    )

    text_col.append(text)

df['text'] = text_col

In [12]:
df.to_csv('train.csv', index=False)
# df[20:].to_csv('train.csv', index=False)
# df[:20].to_csv('test.csv', index=False)
